<a href="https://colab.research.google.com/github/kkosmi/ybigta/blob/master/ML_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ML 과제

과제할 때 꿀팁!

1. 이전 세션들에서의 자료를 찾아본다.
2. 질문방에 질문한다.
3. 구글링한다.
4. 복붙하지 않고 직접 타이핑해서 코드를 완성한다.

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 0. Titanic 데이터 로드 및 훓어보기

In [ ]:
# 여러분이 이전 과제로 만드셨을 FE를 진행한 데이터 보다는 원본 데이터가 조금 더 이번 실습에 적합하기 때문에 원본을 사용하겠습니다.
# 데이터를 로드하고, 잘 불러와진 것인지 첫 5행을 불러와 확인해보세요.

titanic = pd.read_csv('/content/gdrive/MyDrive/titanic_original.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 변수들의 데이터 타입, 결측치, target의 imbalance 여부를 확인해주세요.
titanic.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
titanic.groupby('Survived').size()

Survived
0    549
1    342
dtype: int64

In [ ]:
titanic = titanic.dropna(axis = 0)

In [ ]:
# 편의상 결측치 데이터 및 승객 고유의 데이터('PassengerId','Name', 'Ticket', 'Cabin')는 모두 drop 시킵시다.
# 실제 프로젝트에서는 이렇게 하시면 성능이 낮을 수 있어요!!
dropcol = ['PassengerId','Name', 'Ticket', 'Cabin']
titanic.drop(columns = dropcol,inplace = True)
titanic

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,1,female,38.0,1,0,71.2833,C
3,1,1,female,35.0,1,0,53.1000,S
6,0,1,male,54.0,0,0,51.8625,S
10,1,3,female,4.0,1,1,16.7000,S
11,1,1,female,58.0,0,0,26.5500,S
...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.5542,S
872,0,1,male,33.0,0,0,5.0000,S
879,1,1,female,56.0,0,1,83.1583,C
887,1,1,female,19.0,0,0,30.0000,S


In [ ]:
# categorical feature(Sex, Embarked)에 대해 LabelEncoding을 진행해 주세요.
# 전처리 완료된 DataFrame을 첫 5행을 불러와 확인해보세요
sex_map = {'male':1,'female':0}
titanic['Sex'] = titanic['Sex'].map(sex_map)
titanic.Embarked = titanic.Embarked.replace('C',0)
titanic.Embarked = titanic.Embarked.replace('Q',1)
titanic.Embarked = titanic.Embarked.replace('S',2)
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,1,0,38.0,1,0,71.2833,0
3,1,1,0,35.0,1,0,53.1000,2
6,0,1,1,54.0,0,0,51.8625,2
10,1,3,0,4.0,1,1,16.7000,2
11,1,1,0,58.0,0,0,26.5500,2


In [ ]:
titanic.Pclass.unique()

array([1, 3, 2])

### 1. Train-Test split & 종속변수-설명변수 분할

In [ ]:
# 분할을 위해 필요한 적절한 패키지를 불러와주세요 
# train-test split 및 종속변수-설명변수 분할을 진행해주세요
from sklearn.model_selection import train_test_split

y = titanic.Survived
X = titanic.drop('Survived', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 127)

### 2. Decision Tree를 통한 Classification

In [ ]:
# Decision Tree에서 categorical feature인 'Pclass'를 기준으로 첫 번째 Node(Root Node)가 분리된다고 가정합시다.
# 이 때 Entropy 지표를 통해 Information Gain을 계산할 수 있는 함수(또는 코드)를 작성하고, Information gain을 계산하세요.
# 실습에서 다루지 않은 내용입니다. 코드가 어려우시다면 줄글로만 풀이과정을 작성해주셔도 됩니다.
import math 

def get_entropy(column):
  counts = np.bincount(column)
  prob = counts/len(column)
  
  entropy = 0
  for p in prob:
    if p > 0:
      entropy = entropy + p * math.log(p,2)

  return -entropy

def get_ig(data, split_criterion, target):
  entropy_original = get_entropy(data[target])
  values = data[split_criterion].unique()

  subset1 = data[data[split_criterion] == values[0]]
  subset2 = data[data[split_criterion] == values[1]]
  subset3 = data[data[split_criterion] == values[2]]
  
  to_subtract = 0
  for subset in [subset1,subset2,subset3]:
    prob = (subset.shape[0]/data.shape[0])
    to_subtract += prob * get_entropy(subset[target])

  return entropy_original - to_subtract

#reference:https://towardsdatascience.com/
# titanic data로 Pclass를 분리기준으로 이용했을 때의 entropy를 계산하는 상황만 고려하시면 됩니다.
# 계산에 사용할 데이터는 분할 이전의 0번에서 전처리 완료한 데이터를 기준으로 하겠습니다!
# 올바르게 계산할 시 Information Gain은 0.0097이 나옵니다.


##### <b>줄글 답안</b>
>

In [ ]:
get_ig(titanic, 'Pclass', 'Survived')

0.009762319150193322

In [ ]:
# Decision Tree 모델을 사용하기 위한 적절한 패키지를 불러와 주세요
# 별 다른 parameter 설정 없이 객체를 만들어 주고, 학습을 진행하세요

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

pred = dtree.predict(X_test)

### 3. Graphviz를 통한 시각화

graphviz 설치를 하지 않으셨거나 오류가 난다면 이 부분은 건너뛰셔도 좋습니다

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(# [주석 기호를 지운 뒤 할당한 객체 이름을 적어주세요] ,
                out_file=("titanic_hw.dot"),
                feature_names=X_train.columns,
                class_names=['0','1'],
                rounded=True,
                filled=True)

import graphviz
import os

os.environ["PATH"] += os.pathsep + 'C:/graphviz-2.38/release/bin'

with open("titanic_hw.dot") as f:
    dot_graph = f.read()
    display(graphviz.Source(dot_graph))

### 4. 결과 확인

In [ ]:
# accuracy, f1 score, roc_auc score, confusion matrix, classification report를 모두 불러와주세요
# 불러온 metric(결과 확인 지표)들을 통해 결과를 확인해주세요.
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix,classification_report

print('{:.3f}'.format(accuracy_score(y_test, pred)))
print('{:.3f}'.format(f1_score(y_test, pred)))
print('{:.3f}'.format(roc_auc_score(y_test, pred)))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test,pred))


0.784
0.852
0.710
[[ 6  6]
 [ 2 23]]
              precision    recall  f1-score   support

           0       0.75      0.50      0.60        12
           1       0.79      0.92      0.85        25

    accuracy                           0.78        37
   macro avg       0.77      0.71      0.73        37
weighted avg       0.78      0.78      0.77        37



### 5. GridSearch를 통한 Tree pruning

In [ ]:
# GridSearch를 위한 적절한 패키지를 불러와 주세요
# max_leaf_nodes를 제외한 3가지 이상의 parameter에 대한 GridSearch를 진행해주세요. scoring은 원하시는 지표로 하셔도 됩니다.
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [2,3,4,5,6]  ,
                   'max_features': [2,3,4,5,6,7],
                   'min_samples_split':[1,2,3,4]}

grid_search = GridSearchCV(dtree,param_grid, scoring = 'r2')
grid_search.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                    

In [ ]:
# 최적 parameter 조합 / Validation score / Test score를 확인해 주세요.
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.score(X_test,y_test))

{'max_depth': 2, 'max_features': 6, 'min_samples_split': 4}
-0.14508771929824577
0.2599999999999999


> ### <b>고생하셨습니다~</b>

생각보다 GridSearch의 결과가 안 좋게 나와서 이상하다 생각하신 분들도 있을 수 있습니다. GridSearch 특성상 기존에 정해준 parameter 범위 내에서 찾는 것이기 때문에 애초에 적절한 parameter 후보를 찾지 못했다면 성능이 안 좋을 수도 있습니다. 앞으로 여러 프로젝트 진행해보시면서, parameter 찾는 것에 대한 감도 스스로 익혀보시면 좋을 것 같습니다!